In [1]:
import pandas as pd
import joblib
from decouple import config
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime, timedelta, time
import time 
from sqlalchemy import text
import oracledb
import sys

In [16]:
# Cargar el modelo
modelo = joblib.load('modelo_random_forest_SDC.joblib')

In [2]:
#CONEXIONES
DB_USER=config("USER2_BD_SDC_POSTGRES")
DB_PASSWORD=config("PASS2_BD_SDC_POSTGRES")
DB_NAME="GCTIC_SDC"
DB_PORT="5432"
DB_HOST=config("HOST2_BD_SDC_POSTGRES")
cadena1  = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine1 = create_engine(cadena1)
connection1 = engine1.connect()

In [3]:
tabla = "citas_cext"
col = "citambproconfec"

nuevos_datos = pd.read_sql_query(f"SELECT * FROM {tabla} where {col}>='2024-05-01' and {col}<'2024-05-12 AND grupocupcod IN ('01', '02', '03', '04', '05', '21', '22', '23' ,'24','25') AND tipocitacod IN ('1', '2', '3', '4', '6')'", con=connection1)

In [4]:
nuevos_datos.columns = nuevos_datos.columns.str.upper()

In [6]:
nuevos_datos = nuevos_datos.drop(['AREHOSCOD','REDASISCOD', 'CENASICOD' , 'SERVHOSCOD','ACTCOD','GRUPOCUPCOD', 'CONDTRABCOD','TIPOCITACOD', 'CONDCITACOD', 'MODOTORCITACOD', 'ESTCITCOD'],axis=1)

In [7]:
# Filtrar fechas que están fuera del rango razonable (por ejemplo, antes de 1900 y después de 2100)
def filter_dates(date_series, min_year=1900, max_year=2100):
    date_series = pd.to_datetime(date_series, errors='coerce')
    return date_series[(date_series.dt.year >= min_year) & (date_series.dt.year <= max_year)]

nuevos_datos['MEDNACFEC'] = filter_dates(nuevos_datos['MEDNACFEC'])
nuevos_datos['PACNACFEC'] = filter_dates(nuevos_datos['PACNACFEC'])
nuevos_datos['CITAMBCREFEC'] = filter_dates(nuevos_datos['CITAMBCREFEC'])
nuevos_datos['CITAMBPROCONFEC'] = filter_dates(nuevos_datos['CITAMBPROCONFEC'])

In [8]:
# Extraer solo la hora de la columna CITAMBHORCIT
def extract_time_from_string(time_str):
    try:
        # Si es un objeto datetime, extraer la hora directamente
        if isinstance(time_str, datetime):
            return time_str.time()
        # Si es una cadena de texto, procesarla
        if ' ' in time_str:
            time_str = time_str.split(' ')[1]
        return datetime.strptime(time_str, '%H:%M:%S').time()
    except (ValueError, TypeError):
        return None

nuevos_datos['CITAMBHORCIT'] = nuevos_datos['CITAMBHORCIT'].apply(extract_time_from_string)

In [9]:
nuevos_datos['HORA_CITA'] = nuevos_datos['CITAMBHORCIT'].apply(lambda x: x.hour * 60 + x.minute if pd.notnull(x) else None)

In [10]:
# Calcular la edad del médico y del paciente
nuevos_datos['EDAD_MEDICO'] = (pd.to_datetime('today') - nuevos_datos['MEDNACFEC']).dt.days // 365
nuevos_datos['EDAD_PACIENTE'] = (pd.to_datetime('today') - nuevos_datos['PACNACFEC']).dt.days // 365

# Calcular el tiempo entre la fecha de otorgamiento y la fecha de la cita
nuevos_datos['DIAS_ENTRE_CITA'] = (nuevos_datos['CITAMBPROCONFEC'] - nuevos_datos['CITAMBCREFEC']).dt.days + 1

# Calcular el precedente del paciente
#nuevos_datos['NUM_CITAS'] = nuevos_datos.groupby('PERDOCIDENNUM')['PERDOCIDENNUM'].transform('count')
#nuevos_datos['NUM_DESERCIONES'] = nuevos_datos.groupby('PERDOCIDENNUM')['ESTCITNOM'].transform(lambda x: (x == 'DESERCION').sum())
nuevos_datos['TASA_DESERCION'] = nuevos_datos['NUM_DESERCIONES'] / nuevos_datos['NUM_CITAS']

# Eliminar columnas originales que no serán necesarias
nuevos_datos = nuevos_datos.drop(columns=['MEDNACFEC', 'PACNACFEC', 'CITAMBCREFEC', 'CITAMBPROCONFEC', 'CITAMBHORCIT', 'PERDOCIDENNUM'])

# Verificar y manejar valores faltantes después de todas las transformaciones
nuevos_datos = nuevos_datos.dropna()

# Codificación One-Hot de variables categóricas
nuevos_datos = pd.get_dummies(nuevos_datos, columns=['REDASISDES', 'CENASIDES', 'AREHOSDES', 'SERVHOSDES', 'ACTDES', 'GRUPOCUPNOM', 'CONDTRABNOM', 'TIPOCITANOM', 'CONDCITANOM', 'MODOTORCITANOM'])

# Limpiar nombres de columnas
nuevos_datos.columns = nuevos_datos.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

# Separar características (X) y variable objetivo (y)
X = nuevos_datos.drop(columns=['ESTCITNOM'])
y = nuevos_datos['ESTCITNOM'].apply(lambda x: 1 if x == 'DESERCION' else 0)

In [22]:
import numpy as np

# Obtener los nombres de las características del conjunto de datos de entrenamiento
train_feature_names = X.columns

# Crear un DataFrame con las mismas columnas que el conjunto de datos de entrenamiento
nuevos_datos_alineados = pd.DataFrame(columns=train_feature_names)

# Añadir los nuevos datos al DataFrame alineado
nuevos_datos_alineados = nuevos_datos_alineados.append(nuevos_datos, ignore_index=True)

# Asegurarse de que todas las características faltantes se añadan con valor 0
for feature in train_feature_names:
    if feature not in nuevos_datos_alineados.columns:
        nuevos_datos_alineados[feature] = False

# Eliminar cualquier característica extra en nuevos_datos que no esté en train_feature_names
nuevos_datos_alineados = nuevos_datos_alineados[train_feature_names]

# Crear una máscara para identificar las filas válidas (sin valores nulos)
valid_mask = nuevos_datos_alineados.notnull().all(axis=1)

# Inicializar un array con NaN para las predicciones
prediccion_desercion = np.full(len(nuevos_datos_alineados), np.nan)
probabilidades_desercion = np.full(len(nuevos_datos_alineados), np.nan)

# Realizar las predicciones solo en las filas válidas
if valid_mask.any():
    prediccion_desercion[valid_mask] = modelo.predict(nuevos_datos_alineados[valid_mask])
    probabilidades_desercion[valid_mask] = modelo.predict_proba(nuevos_datos_alineados[valid_mask])[:, 1]

# Mostrar las probabilidades de deserción
print(probabilidades_desercion)


/tmp/ipykernel_1289756/3827877569.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nuevos_datos_alineados = nuevos_datos_alineados.append(nuevos_datos, ignore_index=True)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- TIPOCITANOM_PROCEDIMIENTO
Feature names seen at fit time, yet now missing:
- CENASIDES_PMCONCHUCOS
- CENASIDES_PMOYOTUN
- GRUPOCUPNOM_ENFERMERAO
- GRUPOCUPNOM_OTROS
- GRUPOCUPNOM_PROFESIONAL
- ...


In [17]:
# Asegurarse de que las columnas en los nuevos datos coincidan con las de X_train/X_test
for col in X.columns:
    if col not in nuevos_datos.columns:
        nuevos_datos[col] = 0

nuevos_datos = nuevos_datos[X.columns]

# Realizar las predicciones
prediccion_desercion = modelo.predict(nuevos_datos)[:, 1]
probabilidades_desercion = modelo.predict_proba(nuevos_datos)[:, 1]

# Mostrar las probabilidades de deserción
print(probabilidades_desercion)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- TIPOCITANOM_PROCEDIMIENTO
Feature names seen at fit time, yet now missing:
- CENASIDES_PMCONCHUCOS
- CENASIDES_PMOYOTUN
- GRUPOCUPNOM_ENFERMERAO
- GRUPOCUPNOM_OTROS
- GRUPOCUPNOM_PROFESIONAL
- ...
